## Formatera data från SCAAR så output kan laddas till DosReg

Gå till https://www.ucr.uu.se/swedeheart/ och logga in med Siths-kort\
Gå till "Rapporter"\
Välj "Export till Excel Angio-PCI"\
Rapportdatum: T.ex 2024-01-01 - 2025-01-01\
Angio/PCI: Enbart angio\
Procedur/Segment: Procedur\
Klicka på "Beställ" och ladda ner resulterande excel-fil.\
Öppna excel-fil, radera första 3 kolumnerna "Personnr eller motsv", "Typ av personnummer" och "Födelsedatum" och spara till input_data.


In [ ]:
import pandas as pd

# Läs in data från SCAAR

SCAAR_data_path = "C:/Projekt/GIT/rvbrtg/Data/input_data/Angio_2024.xlsx"
data = pd.read_excel(SCAAR_data_path)

#data.head()

#Minska ner tabellen och döp om kolumner

data_subset = data[["Kön", "Ålder vid procedur", "Datum för procedur", "Längd (cm)", "Vikt (kg)", "Angiograför", "Punktionställe", "Labnamn", "Stråldos (µGym2)", "Genomlysningstid (h:mm:ss)", ]].copy()

data_subset.columns = ["Sex", "Age", "Datum", "Length_cm", "Weight_kg", "Operator", "Accesspoint", "Lab", "KAP_uGym2", "Fluorotime_h_mm_ss"]

#data_subset.head()

#Ersätt , med . samt byt från sträng till float och konvertera från uGym2 till Gycm2
data_subset["KAP_uGym2"] = data_subset["KAP_uGym2"].replace(',','.',regex=True).astype(float)
data_subset["KAP_uGym2"] = data_subset["KAP_uGym2"] * 0.01

data_subset.rename(columns = {"KAP_uGym2":"KAP_Gycm2"}, inplace = True)

#data_subset.dtypes
#data_subset.head()

In [ ]:
#Kolla datatyper. Är KAP numeriskt eller en sträng

data_subset.dtypes

In [ ]:
#filtrera ut data

data_subset_weight = data_subset[(data_subset["Weight_kg"] > 60) & (data_subset["Weight_kg"] < 90)]

#data_subset_weight.head()

data_subset_U601 = data_subset_weight[data_subset_weight["Lab"] == "Lab 1"]

data_subset_U601['DAPdiff'] = (
        data_subset_U601["KAP_Gycm2"] - data_subset_U601.loc[:, "KAP_Gycm2"].median()
).abs()
data_subset_U601.sort_values('DAPdiff', inplace=True, ignore_index=True)


data_subset_U602 = data_subset_weight[data_subset_weight["Lab"] == "Lab 2"]

data_subset_U602['DAPdiff'] = (
        data_subset_U602["KAP_Gycm2"] - data_subset_U602.loc[:, "KAP_Gycm2"].median()
).abs()
data_subset_U602.sort_values('DAPdiff', inplace=True, ignore_index=True)


#data_subset_U601.head()

In [ ]:
#Printa ut antal per kön och medelvärde för KAP för att skriva in i DosReg-mall

print(data_subset_weight.groupby("Sex").size())

print(data_subset_weight.groupby("Sex").mean(numeric_only = True))

In [ ]:
# Ensure 'Fluorotime_h_mm_ss' is in string format before converting to timedelta
data_subset_weight["Fluorotime_h_mm_ss"] = data_subset_weight["Fluorotime_h_mm_ss"].astype(str)

# Convert 'Fluorotime_h_mm_ss' to seconds for numerical aggregation
data_subset_weight["Fluorotime_seconds"] = pd.to_timedelta(data_subset_weight["Fluorotime_h_mm_ss"]).dt.total_seconds()

# Group by 'Operator' and calculate mean values for specified columns
mean_values = data_subset_weight.groupby("Lab").median(numeric_only=True)[["Age", "Length_cm", "Weight_kg", "KAP_Gycm2"]]

# Add mean of 'Fluorotime_seconds' to the grouped data
mean_values["Fluorotime_seconds"] = data_subset_weight.groupby("Lab")["Fluorotime_seconds"].mean()

# Convert 'Fluorotime_seconds' back to h:mm:ss format
mean_values["Fluorotime_h_mm_ss"] = pd.to_timedelta(mean_values["Fluorotime_seconds"], unit='s')

# Drop the intermediate 'Fluorotime_seconds' column
mean_values = mean_values.drop(columns=["Fluorotime_seconds"])

# Display the result
print(mean_values)